In [19]:
from model.AnoGAN import AnoGAN
from fastprogress import master_bar, progress_bar
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torchmetrics.functional.classification import binary_auroc, binary_accuracy
import torch

In [16]:
dataset = CIFAR10(root='data/', download=True, transform=ToTensor())
test_dataset = CIFAR10(root='data/', train=False, transform=ToTensor())

label = 1
dataset.targets = (torch.Tensor(dataset.targets) == label).type(torch.float)
test_dataset.targets = (torch.Tensor(test_dataset.targets) == label).type(torch.float)

# dataset = torch.utils.data.Subset(dataset, (dataset.targets == label).nonzero(as_tuple=True)[0])
test_dataset = torch.utils.data.Subset(test_dataset, range(1250))
batch_size = 250
train_loader = DataLoader(dataset, batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size, num_workers=2)

Files already downloaded and verified


In [13]:
anogan = AnoGAN()

In [11]:
epochs = 20
mb = master_bar(range(epochs))
AUROC_converg = []
ACC_converg = []
total_acc = []
for epoch in mb:
    for j,(image,y) in enumerate(progress_bar(train_loader, parent = mb)):
        auroc, acc, g_loss, d_loss, fake, real = anogan.train(image, y, verbose = True)
        mb.child.comment = "gen_loss: {} dis_loss: {} fake: {} real: {}".format(g_loss, d_loss, fake, real)
    
    batch_auroc = []
    batch_acc = []
    for j,(batch,y) in enumerate(progress_bar(test_loader, parent = mb)):  
        preds, auroc, acc, g_loss, d_loss, fake, real = anogan.predict(batch, y, verbose = True)
        batch_auroc.append(float(auroc.data))
        batch_acc.append(float(acc.data))
        total_acc.append(float(binary_accuracy(preds, y).data))
    AUROC_converg.append(sum(batch_auroc) / len(batch_auroc))
    ACC_converg.append(sum (batch_acc) / len (batch_acc))
    mb.main_bar.comment = "AUROC: {} ACC:{}".format(sum(batch_auroc) / len(batch_auroc), sum(batch_acc) / len(batch_acc))
    
    mb.write('Epoch {}: Total Accuracy: {} Focal Accuracy: {} AUROC: {}'.format(epoch, 
                                                                                   sum(total_acc) / len(total_acc),
                                                                                   sum(batch_acc) / len(batch_acc),
                                                                                   sum(batch_auroc) / len(batch_auroc)))

In [12]:
torch.save(anogan, "AnoGAN_model_20_full.pth")

In [ ]:
AUROC_converg

In [ ]:
def VotingClassifier(input1, input2, input3, threshold = 0.5):
    return torch.mode(torch.where(torch.concat((input1, input2, input3), dim = 1)>threshold, 1.,0.)).values

In [ ]:
def AverageClassifier(input1, input2, input3, threshold = 0.5):
    return torch.where(torch.mean(torch.concat((input1, input2, input3), dim = 1), dim =1)>threshold, 1.,0.)

In [ ]:
def LinearEnsemble(nn.Module):
    def __init__(self, anoGAN, secondModel):
        super(MyEnsemble, self).__init__()
        self.anoGAN = anoGAN.D.forward
        self.secondModel = modelB
        self.classifier = nn.Linear(in_features, 1) #define accordingly
        self.relu = nn.ReLU()

    def forward(self, x1, x2):
        x1 = self.firstModel(x1)
        x2 = self.secondModel(x2)
        x = torch.cat((x1, x2), dim=1)
        x = self.classifier(self.relu(x))
        return x

In [ ]:
input1 = torch.rand(250, 1)
input2 = torch.rand(250, 1)
input3 = torch.rand(250, 1)

In [14]:
torch.load("C:\\Users\\galli\\Documents\\GitHub\\cs7643_final_project\\ensemble_ad\\AnoGAN\\AnoGAN_model_0.pt")

AnoGAN(
  (loss): BCELoss()
  (G): Generator(
    (layer_0): Sequential(
      (0): ConvTranspose2d(100, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer_1): Sequential(
      (0): ConvTranspose2d(1024, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer_2): Sequential(
      (0): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer_3): Sequential(
      (0): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
 